# Call library

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm 
import json 
from utils import read_json 
from dataset import MengziT5Dataset

# Data Preprocessing

In [ ]:
DATA_TRAIN_PATH = "data/train.json"
DATA_DEV_PATH = "data/dev.json"

valid_data = read_json(DATA_DEV_PATH)
print("First valid data: ", valid_data[0])
train_data = read_json(DATA_TRAIN_PATH;)
print("First train data: ", train_data[0])


Reading JSON file: 984it [00:00, 75261.59it/s]

First valid data:  {'context': '年基准利率4.35%。 从实际看,贷款的基本条件是: 一是中国大陆居民,年龄在60岁以下; 二是有稳定的住址和工作或经营地点; 三是有稳定的收入来源; 四是无不良信用记录,贷款用途不能作为炒股,赌博等行为; 五是具有完全民事行为能力。', 'answer': '年基准利率4.35%', 'question': '2017年银行贷款基准利率', 'id': 0}


# Model Retrieval

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

checkpoint = "Langboat/mengzi-t5-base"

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint) 

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
